Data is made through roboflow. images are collected through openimagesv6. They are annotated through roboflow and imported to .tfrecord format. dont run. it has been run on cpu. there is some problem with running on gpu. image is resized to 416x416

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection

/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection


In [ ]:
#clone the tensorflow model repo
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 72179, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 72179 (delta 3), reused 9 (delta 2), pack-reused 72168
Receiving objects: 100% (72179/72179), 578.97 MiB | 10.65 MiB/s, done.
Resolving deltas: 100% (51068/51068), done.
Checking out files: 100% (3025/3025), done.


In [ ]:
#installing other libraries
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

%cd /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/models/research
#compiling proto files
!protoc object_detection/protos/*.proto --python_out=.

%cp object_detection/packages/tf2/setup.py .

!python -m pip install --use-feature=in-tree-build .
#running object detection model builder files to check its working or not
!python '/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/models/research/object_detection/builders/model_builder_tf2_test.py'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.6).
python-pil is already the newest version (5.1.0-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/models/research
Processing /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=21913350 sha256=366426c462624a2dad003928f6f691894a3eeb44c4cefafe88279e76aa08ddb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhxc3rm0/wheels/00/3e/85/c438e993c8fbb80934e74e79199dfa619f797e5c751b9adc26
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstal

In [ ]:
#importing some libraries
import matplotlib
import matplotlib.pyplot as plt
import tarfile
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
#opencv-contrib-python and opencv-python has to be same level or always upgraded alse the the trainging time model will throw error
!python -m pip install --upgrade opencv-contrib-python
!python -m pip install --upgrade opencv-python

In [ ]:
# #loading model from tensorflow hub ###not required
# module_handle = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
# model = hub.load(module_handle)

In [ ]:
# #checking keys of the model whether those keys are for classification, object detection ##not required
# print(hub.load(module_handle).signatures.keys())
# model.signatures['serving_default']

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection

/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection


In [ ]:
# !unzip Helmet.v5i.tfrecord.zip

In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection'

In [ ]:
#loading and checking train tfrecord files
train_dataset = tf.data.TFRecordDataset("train/Helmet.tfrecord")
for x in train_dataset.take(5):
  print(x)

tf.Tensor(b'\n\xf4\x98\x02\n\xd8\x94\x02\n\rimage/encoded\x12\xc5\x94\x02\n\xc1\x94\x02\n\xbd\x94\x02\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01\xa0\x01\xa0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\x

In [ ]:
# %mkdir 'pre-trained models'
# %mkdir 'ssd_mobilenet_v2_320*320'

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/pre-trained-models

/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection/pre-trained-models


In [ ]:
#download pre-trained weights and config file. It can be done through copying the file from models/research/object_detection/confids/tf2/model_name
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
!wget {download_tar}

--2022-04-27 10:10:10--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.196.128, 2607:f8b0:4001:c1a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.196.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  52.2MB/s    in 0.4s    

2022-04-27 10:10:11 (52.2 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection'

In [ ]:
#decompressing model files
tar = tarfile.open('ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
tar.extractall()
tar.close()

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection

/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection


In [ ]:
#maintaining pipeing config path and model directory
pipeline_file_path = "pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"
model_path = "pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
num_steps = 1000
num_eval_steps = 500

**
changes to made in pipeline config file
1. change the number of classes
2. check the size of the image u want
3. always maintain checkpoints as checkpoint0 something because after downloading from the github repo it has simply checkpoint name and it will not run and throw framework error
4. checkpoint files has big names just put ckpt-0 where 0 is the checkpoint in the fine-tune checkpoint. if not done, same tensorflow framework error
5. always maintain a directory structure
6. new checkpoint files is created every 1000 epochs
7. before training check the pipeling.config file and make changes to num_classes to 1 because our model is going to predict 1 class only.
8. check the fine_tune_checkpoint part and make the changes like detection if your work is for detection and change the batch size as per you req but only to be factor of 2
9. data augmentation changes can also be done there.
10. add train and test label map  and tfrecord file paths for training.
11. changes can be made to num of steps or eval steps too if you have validation files.



**

In [ ]:
#checking opencv is of same version
pip list | grep opencv

opencv-contrib-python         4.5.5.64
opencv-python                 4.5.5.64
opencv-python-headless        4.5.5.64


In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/ColabNotebooks/Helmet-Detection'

first, mobilenetv2 300x300 model was utilized but it was not able to display bounding boxed on inference or there is some bug in the model. older models are preferred more in detection. currently ssd mobilenetv2 fpnlite model is used. it has mobilenetv2 as a backbone to get high level features and ssd acts as Single Shot MultiBox Detector to do detection tasks|. it has fpn lite network which gives feature maps at multiple levels from a single image

In [ ]:
# training the model
#image is process to 300x300
!python "models/research/object_detection/model_main_tf2.py" \
    --pipeline_config_path={pipeline_file_path} \
    --model_dir={model_path} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2022-04-27 11:25:44.819016: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0427 11:25:44.820771 140072469415808 cross_device_ops.py:1386] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0427 11:25:44.825241 140072469415808 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 1000
I0427 11:25:44.833887 140072469415808 config_util.py:552] Maybe overwriting train_steps: 1000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0427 11:25:44.834117 140072469415808 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0427 11:25:45.017541 140072469415808 deprecation.py:343] From /usr/local/

In [ ]:
#Recover our saved model with the latest checkpoint:
pipeline_config = pipeline_file_path
#Put the last ckpt from training in here, don't use long pathnames:
model_dir = "pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/training/ckpt-3"

#configuring the pipeline config file
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']

#building custome model from tht checkpoint
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore last checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)

#restoring model from that checkpoint
ckpt.restore(model_dir)

#Function perform detection of the object on image in tensor format: 
def get_model_detection_function(model):
  """Get a tf.function for detection."""
  
  @tf.function
  def detect_fn(image):
    """Detect objects in image."""
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn
  
#Define function which performs detection: 
detect_fn = get_model_detection_function(detection_model)

In [ ]:
#category index for generating categories of labels in the image which we labelled and annotate
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
print(category_index)

{1: {'id': 1, 'name': 'Helmet'}}


In [ ]:
#it takes a little longer on the first run and then runs at normal speed. 
import random

#Define utility functions for presenting the results:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.
  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.
  bytesio putting the image into memory 
  Args:
    path: the file path to the image
  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def inferencing(image_path):
  #Store test images in nmpy array:
  image_np = load_image_into_numpy_array(image_path)

  #Convert images to tensor form:
  input_tensor = tf.convert_to_tensor(
      np.expand_dims(image_np, 0), dtype=tf.float32)

  #Perform detection on the image in tensor format:
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  #Visualize the detection boxes on the image:
  #offset required for moving center of the anchor boxes 
  label_id_offset = 1
  image_np_with_detections = image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.20,#0.5,#0.5
        agnostic_mode=False,
  )

  plt.figure(figsize=(30, 30))
  plt.imshow(image_np_with_detections)
  plt.show()

In [ ]:
inferencing("test-images/3730005070_ac3deaf23f_o.jpg")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
inferencing("test-images/3972267404_8ef9c10920_o.jpg")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
inferencing("test-images/6912178570_a293c42e7d_o.jpg")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
inferencing("test-images/9665857337_78625e7a31_o.jpg")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(detections)

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.01298583, 0.64277124, 0.27974153, 0.9071485 ],
        [0.20320399, 0.14892924, 0.442011  , 0.3115254 ],
        [0.02469127, 0.66559684, 0.41472012, 0.8898393 ],
        [0.19460258, 0.15989025, 0.34904805, 0.3097825 ],
        [0.08030313, 0.4469551 , 0.2635288 , 0.5680394 ],
        [0.01329858, 0.27105853, 0.06307832, 0.36511216],
        [0.18812498, 0.13466638, 0.5185083 , 0.34056088],
        [0.07592815, 0.39144748, 0.36324373, 0.6356432 ],
        [0.06204027, 0.5470835 , 0.12472706, 0.64450574],
        [0.8887499 , 0.282873  , 0.9917769 , 0.49093857],
        [0.6190335 , 0.6830508 , 0.79735595, 0.7855938 ],
        [0.46430287, 0.3619256 , 0.82782924, 0.5669006 ],
        [0.38020885, 0.56457144, 0.7741002 , 0.7925052 ],
        [0.7983437 , 0.6328281 , 0.9862861 , 0.74173   ],
        [0.23743698, 0.48268017, 0.34109482, 0.58381724],
        [0.49567717, 0.21734181, 0.6328537 , 0.36606   ],